# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
openai = OpenAI()

API key looks good so far


In [8]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
system_prompt = "You are a tool that helps a user to understand what lines in a program do. Respond in markdown."

In [10]:
# Get gpt-4o-mini to answer, with streaming
def stream_code_help_openai(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
stream_code_help_openai(question)

Certainly! Let's break down the code piece by piece:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation:

1. **Set Comprehension**: 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 
   - It iterates over each `book` in the collection `books`.

2. **Extracting Author**: 
   - For each `book`, it attempts to get the value associated with the key `"author"` using `book.get("author")`.
   - The `get` method is used here because it returns `None` if the key does not exist, which prevents a `KeyError`.

3. **Filtering**: 
   - The `if book.get("author")` part filters the results. 
   - Only those `book` entries are included where the `"author"` key exists and its value is truthy (i.e., not `None`, empty, or `False`).

4. **Creating a Set**:
   - The result of the set comprehension is a set of unique authors (since sets automatically handle duplicates).

5. **Yielding Results**:
   - The `yield from` statement is used in a generator. 
   - It allows the generator to yield each item from the set (i.e., each distinct author name) one at a time.
   - This is useful in cases where you want to produce values iteratively and efficiently, particularly in a loop.

### Why It's Useful:

- **Avoid Duplicates**: By using a set, the code ensures that each author is only yielded once, even if multiple books by the same author exist in the `books` collection.
- **Efficient Iteration**: The `yield from` construct allows for efficient iteration over the collection of authors without needing to explicitly loop through them in the generator function.

### Example Context:

Imagine you are processing a list of books, and you want to get a unique list of authors from them. This code snippet helps achieve that efficiently and neatly by combining filtering, extraction, and yielding of results.

### Summary:

In summary, this code constructs a set of unique authors from a list of `book` dictionaries and yields each unique author, thereby efficiently generating a stream of author names without duplicates.

In [14]:
# Get Llama 3.2 to answer
def stream_code_help_llama(question):
    stream =ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
stream_code_help_llama(question)

**Code Explanation**
===============

The given code uses Python's generator syntax to extract the authors of books from a list of dictionaries.

python
yield from {book.get("author") for book in books if book.get("author")}


Here's a breakdown:

*   `for book in books`: This part iterates over each dictionary (`book`) in the `books` list.
*   `if book.get("author")`: This condition filters out dictionaries that don't have an "author" key.
*   `{book.get("author") for book in books if book.get("author")}`: This is a generator expression, which creates a sequence of values. In this case, it yields the author of each dictionary that passes the filter.

**How it works**
----------------

1.  The `yield from` statement tells Python to delegate the iteration over the sequence to an inner iterator (the generator expression).
2.  The generator expression iterates over each book in the list.
3.  If a book has an "author" key, its value is yielded.

**Example Use Case**
--------------------

Suppose you have the following `books` list:

python
books = [
    {"title": "Book A", "author": "John Doe"},
    {"title": "Book B", "author": "Jane Smith"},
    {"title": "Book C"}
]


If you run the given code, it will yield the authors of books with an author:

python
[
    "John Doe",
    "Jane Smith"
]


**Benefits**
------------

Using generator expressions and `yield from` provides several benefits, including:

*   Improved memory efficiency: Generators don't load all values into memory at once.
*   Easier to implement recursive functions or infinite sequences.

Overall, this code snippet is a concise way to extract specific data from a list of dictionaries while filtering out unwanted items.